In [3]:
import numpy as np
import matplotlib.pyplot as plt
!pip install tensorflow
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Dropout, concatenate, Flatten, UpSampling2D,Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


/home/ubuntu/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/keras/backend.py:401: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [4]:
X_train = np.load('X_train_final.npy')
Y_train = np.load('Y_train_final.npy')
[m, n_h, n_w, n_c] = X_train.shape

In [79]:
X_val = np.load('X_val.npy')
Y_val = np.load('Y_val.npy')
Yuncertain = np.unique(np.where(Y_val==-1))
Y_val = np.delete(Y_val, Yuncertain, 0)
X_val = np.delete(X_val, Yuncertain, 0)

In [5]:
Y_train = np.where(Y_train == -1, 0, Y_train)
Y_val = np.where(Y_val == -1, 0, Y_val)

In [106]:
val_fire = np.unique(np.where(X_val[:,:,:,11]==1))
X_val_fire = X_val[val_fire]
Y_val_fire = Y_val[val_fire]

(825, 32, 32, 12)

In [6]:
# Evaluation Metrics : Dice Coefficient and Intersection over Union

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

In [7]:
# Contraction Path
s=1

X = Input((n_h, n_w, n_c))

dropout = 0.1

c1 = Conv2D(16, (3,3), strides =(s,s), padding='same', activation = 'relu', kernel_initializer = 'he_normal')(X)
c1 = Dropout(dropout)(c1)
c1 = Conv2D(16, (3,3), strides =(s,s), padding = 'same', activation='relu', kernel_initializer = 'he_normal')(c1)
c1 = BatchNormalization(axis=3)(c1)
c1 = Activation('relu')(c1)
p1 = MaxPooling2D((2,2))(c1)
    
c2 = Conv2D(32, (3,3), strides =(s,s), padding='same', activation = 'relu', kernel_initializer = 'he_normal')(p1)
c2 = Dropout(dropout)(c2)
c2 = Conv2D(32, (3,3), strides =(s,s), padding = 'same', kernel_initializer = 'he_normal')(c2)
c2 = BatchNormalization(axis=3)(c2)
c2 = Activation('relu')(c2)
p2 = MaxPooling2D((2,2))(c2)
   
c3 = Conv2D(64, (3,3), strides =(s,s), padding='same', activation = 'relu', kernel_initializer = 'he_normal')(p2)
c3 = Dropout(dropout)(c3)
c3 = Conv2D(64, (3,3), strides =(s,s), padding = 'same', kernel_initializer = 'he_normal')(c3)
c3 = BatchNormalization(axis=3)(c3)
c3 = Activation('relu')(c3)
p3 = MaxPooling2D((2,2))(c3)
    
c4 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(p3)
c4 = Dropout(dropout)(c4)
c4 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(c4)
c4 = BatchNormalization(axis=3)(c4)
c4 = Activation('relu')(c4)  
    
 # Expansion path

u5 = UpSampling2D(size = (2,2))(c4)
u5 = concatenate([u5, c3])
c5 = Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u5)
c5 = Dropout(dropout)(c5)
c5 = Conv2D(64, (3,3), kernel_initializer='he_normal', padding='same')(c5)
c5 = BatchNormalization(axis=3)(c5)
c5 = Activation('relu')(c5)                                                                        
    
u6 = UpSampling2D(size = (2,2))(c5)
u6 = concatenate([u6, c2])
c6 = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = Dropout(dropout)(c6)
c6 = Conv2D(32, (3,3), kernel_initializer='he_normal', padding='same')(c6)
c6 = BatchNormalization(axis=3)(c6)
c6 = Activation('relu')(c6) 
    
u7 = UpSampling2D(size = (2,2))(c6)
u7 = concatenate([u7, c1])
c7 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = Dropout(dropout)(c7)
c7 = Conv2D(16, (3,3), kernel_initializer='he_normal', padding='same')(c7)
c7 = BatchNormalization(axis=3)(c7)
c7 = Activation('relu')(c7)                                                                        
                                                        
    
outputs = Conv2D(1, (1,1), activation = 'sigmoid')(c7)

In [8]:
model = Model(inputs = X, outputs = outputs)

In [6]:
from tensorflow.keras.optimizers import Adam

In [7]:
epochs = 100
learning_rate = 1e-4
decay_rate = learning_rate/epochs
batch_size=64
opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=decay_rate, amsgrad=False)

/home/ubuntu/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [139]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef, iou])

In [140]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 12)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   1744        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 32, 32, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   2320        dropout[0][0]                    
______________________________________________________________________________________________

In [141]:
history = model.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/100
170/170 [==============================] - 23s 129ms/step - loss: 0.0620 - dice_coef: 0.3585 - iou: 0.2199
Epoch 2/100
170/170 [==============================] - 22s 129ms/step - loss: 0.0621 - dice_coef: 0.3605 - iou: 0.2214
Epoch 3/100
170/170 [==============================] - 22s 129ms/step - loss: 0.0617 - dice_coef: 0.3637 - iou: 0.2238
Epoch 4/100
170/170 [==============================] - 22s 130ms/step - loss: 0.0614 - dice_coef: 0.3660 - iou: 0.2255
Epoch 5/100
170/170 [==============================] - 22s 128ms/step - loss: 0.0612 - dice_coef: 0.3688 - iou: 0.2276
Epoch 6/100
170/170 [==============================] - 22s 129ms/step - loss: 0.0612 - dice_coef: 0.3668 - iou: 0.2262
Epoch 7/100
170/170 [==============================] - 22s 130ms/step - loss: 0.0612 - dice_coef: 0.3705 - iou: 0.2288
Epoch 8/100
170/170 [==============================] - 22s 128ms/step - loss: 0.0610 - dice_coef: 0.3690 - iou: 0.2279
Epoch 9/100
170/170 [===========================

In [144]:
V2_100 = history.history

In [145]:
np.save('V2_100',V2_100)

In [511]:
model.evaluate(X_val_fire, Y_val_fire)

26/26 [==============================] - 0s 15ms/step - loss: 0.1391 - dice_coef: 0.2577 - iou: 0.1495


[0.13906754553318024, 0.2577064037322998, 0.1494782418012619]

In [147]:
model.save('UnetV2_100')

INFO:tensorflow:Assets written to: UnetV2_100/assets
